In [1]:
import pyfeats
from typing import *
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import os
from utils import *

In [2]:
llf = LowLevelFeatureExtractor(function=pyfeats.glcm_features, 
                               params={'ignore_zeros': True}, 
                               features_set=['features_mean', 'features_range'])

# root_folder = "C:\\Users\\trong\\Documents\\skin_data"
root_folder = "/mnt/c/Users/trong/Documents/skin_data" 

In [3]:
# Define image transformations
transform = transforms.Compose([
    transforms.Resize((128, 128)),  # Resize images to 128x128
    transforms.Grayscale(num_output_channels=1),    # Convert to grayscale
    ToNumpy(),  # Convert to tensor
])

In [4]:
train_dataset = CSVImageMetadataDataset(csv_file='./data/vaynen_train_linux.csv', root_dir=root_folder, transform=transform)
train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=4)   

test_dataset = CSVImageMetadataDataset(csv_file='./data/vaynen_test_linux.csv', root_dir=root_folder, transform=transform)
test_dataloader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=4)

# Check one batch
images, metadata, labels = next(iter(train_dataloader))
print(images.shape)  # Example: torch.Size([32, 3, 128, 128])
print(metadata.shape)  # Example: torch.Size([32, 6])  -> 6 metadata features
print(labels.shape)  # Example: torch.Size([32])

torch.Size([32, 128, 128])
torch.Size([32, 6])
torch.Size([32])


In [3]:
llf.get_features_size()

28

In [8]:
import numpy as np

# Simulating a batch tensor (random example)
batch_tensor = np.random.randn(32, 1, 128, 128)  # Shape (32,1,128,128)

# Process each image separately
processed_images = []
for img in batch_tensor:  # Automatically iterates over axis 0
    processed_img = img * 2  # Example operation: scaling by 2
    processed_images.append(processed_img)

# Stack them back to maintain original shape
final_batch_tensor = np.stack(processed_images, axis=0)

print(final_batch_tensor.shape)  # Output: (32, 1, 128, 128)


(32, 1, 128, 128)


In [ ]:
model = SimpleNeuralNetwork(inputs = llf.get_features_size())

# Check the model's architecture
print(model)

In [ ]:
train_model(model, train_dataloader, epochs=100, llf=llf ,features_set=llf.function.__name__, )

In [ ]:
evaluate_model(model, test_dataloader, features_set=llf.function.__name__)

In [ ]:
from PIL import Image

image = Image.open("/mnt/c/Users/trong/Documents/skin_data/train/12/image_uuid_000e1f2d-7442-4cdb-89d0-b8350ecdebbb_created_by_haianh.31297_MTwJXvl.png")
gray_image = image.convert("L")

gray_image_array = np.array(gray_image)

In [ ]:
import torch
from torchvision import models

model = models.vgg16(pretrained=True)
optimizer = torch.optim.SGD(model.parameters(), lr=0.001)

# Train the model...

# Save the model using PyTorch's built-in function
torch.save(model.state_dict(), 'my_model.pth')

In [ ]:
type(torch.Tensor(llf(transform(image))))

In [ ]:
for llf_param in param_list[-6:]:
    llf = LowLevelFeatureExtractor(**llf_param)
    print(f'{llf.function.__name__} -> {llf(gray_image_array)}')

In [ ]:
param_list[0]